In [1]:
import keras as keras
keras.__version__

'2.4.3'

In [2]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [3]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 불러오기 및 분석

In [4]:
train_data = pd.read_excel("gradnet_train_v2.xlsx")
print('전체 학습 데이터의 수 :',(len(train_data)))

전체 학습 데이터의 수 : 13578


In [5]:
#결측치 확인
train_data.isnull().sum()

id           0
file_name    0
text         4
label        0
dtype: int64

In [6]:
#결측치 삭제
train_data = train_data.dropna()

In [7]:
train_data.head()

,id,file_name,text,label
0,6205,399,ÉTS TOEFL IBT.\nTest Taker Score Report\nETS S...,IBT
1,6203,3926,ETS)\nTOEFL İBT.\nTest Taker Score Report\nNam...,IBT
2,6110,2062,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT
3,6045,2020321275,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT
4,6255,600,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT


In [8]:
# 시험명(label) 종류와 개수 확인
train_data['label'].value_counts()

성적증명서    4886
졸업증명서    4654
TOEIC    2286
TEPS      487
TOPIK     362
IBT       335
IELTS     194
PBT       170
기타        151
졸업+성적      49
Name: label, dtype: int64

In [9]:
#label 열 복사
train_data['label2'] = train_data['label']

In [10]:
train_data

,id,file_name,text,label,label2
0,6205,399,ÉTS TOEFL IBT.\nTest Taker Score Report\nETS S...,IBT,IBT
1,6203,3926,ETS)\nTOEFL İBT.\nTest Taker Score Report\nNam...,IBT,IBT
2,6110,2062,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT,IBT
3,6045,2020321275,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT,IBT
4,6255,600,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,IBT,IBT
...,...,...,...,...,...
13573,1146,부산대학교_1A1OA01002_0_U_대학 졸업(예정)증명서,제 N2020123316 호\n졸업증명 서\n성\n명\n김상온\n생 년월 일\n19...,졸업증명서,졸업증명서
13574,4247,연세대학교_1A1AG01003_0_U_대학 졸업(예정)증명서,4 Docu. Verification No. : XD6A-4C8D-C36E-41A2...,졸업증명서,졸업증명서
13575,1236,서울대학교_1A1JA01266_0_U_대학 졸업(예정)증명서,[ 원본대조번호 : 07A1-5D18-0719-2159 ]\n제 0272059 호\...,졸업증명서,졸업증명서
13576,248,Ningbo Polytechnic_1C1JA01002_1_U_대학 졸업(예정)증명서,公 证 书\n中华人民共和国吉林省安图县公证处\n普通高等学校\n毕业证书\n学生 韩情情\...,졸업증명서,졸업증명서


In [11]:
train_data['label'] = train_data['label'].replace('TOEIC', '0')
train_data['label'] = train_data['label'].replace('IBT', '1')
train_data['label'] = train_data['label'].replace('TEPS', '1')
train_data['label'] = train_data['label'].replace('IELTS', '1')
train_data['label'] = train_data['label'].replace('TOPIK', '1')
train_data['label'] = train_data['label'].replace('PBT', '1')
train_data['label'] = train_data['label'].replace('졸업증명서', '1')
train_data['label'] = train_data['label'].replace('성적증명서', '1')
train_data['label'] = train_data['label'].replace('졸업+성적', '1')
train_data['label'] = train_data['label'].replace('기타', '1')
train_data

,id,file_name,text,label,label2
0,6205,399,ÉTS TOEFL IBT.\nTest Taker Score Report\nETS S...,1,IBT
1,6203,3926,ETS)\nTOEFL İBT.\nTest Taker Score Report\nNam...,1,IBT
2,6110,2062,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
3,6045,2020321275,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
4,6255,600,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
...,...,...,...,...,...
13573,1146,부산대학교_1A1OA01002_0_U_대학 졸업(예정)증명서,제 N2020123316 호\n졸업증명 서\n성\n명\n김상온\n생 년월 일\n19...,1,졸업증명서
13574,4247,연세대학교_1A1AG01003_0_U_대학 졸업(예정)증명서,4 Docu. Verification No. : XD6A-4C8D-C36E-41A2...,1,졸업증명서
13575,1236,서울대학교_1A1JA01266_0_U_대학 졸업(예정)증명서,[ 원본대조번호 : 07A1-5D18-0719-2159 ]\n제 0272059 호\...,1,졸업증명서
13576,248,Ningbo Polytechnic_1C1JA01002_1_U_대학 졸업(예정)증명서,公 证 书\n中华人民共和国吉林省安图县公证处\n普通高等学校\n毕业证书\n学生 韩情情\...,1,졸업증명서


In [12]:
train_data['label'].value_counts()

1    11288
0     2286
Name: label, dtype: int64

## 데이터 전처리

In [13]:
import re
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [14]:
def preprocessing(text, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    text = re.sub("[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ",text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = text.lower().split()

    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [15]:
train_data

,id,file_name,text,label,label2
0,6205,399,ÉTS TOEFL IBT.\nTest Taker Score Report\nETS S...,1,IBT
1,6203,3926,ETS)\nTOEFL İBT.\nTest Taker Score Report\nNam...,1,IBT
2,6110,2062,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
3,6045,2020321275,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
4,6255,600,ETS TOEFL iBT.\nTest Taker Score Report\nName:...,1,IBT
...,...,...,...,...,...
13573,1146,부산대학교_1A1OA01002_0_U_대학 졸업(예정)증명서,제 N2020123316 호\n졸업증명 서\n성\n명\n김상온\n생 년월 일\n19...,1,졸업증명서
13574,4247,연세대학교_1A1AG01003_0_U_대학 졸업(예정)증명서,4 Docu. Verification No. : XD6A-4C8D-C36E-41A2...,1,졸업증명서
13575,1236,서울대학교_1A1JA01266_0_U_대학 졸업(예정)증명서,[ 원본대조번호 : 07A1-5D18-0719-2159 ]\n제 0272059 호\...,1,졸업증명서
13576,248,Ningbo Polytechnic_1C1JA01002_1_U_대학 졸업(예정)증명서,公 证 书\n中华人民共和国吉林省安图县公证处\n普通高等学校\n毕业证书\n学生 韩情情\...,1,졸업증명서


In [16]:
# 전체 학습데이터 전처리
clean_train_texts = []
for text in train_data['text']:
    clean_train_texts.append(preprocessing(text, remove_stopwords = True))

In [17]:
# 전처리한 데이터 출력
clean_train_texts[112]

'ets toefl ibt test taker score report name jeong moon kyung last family surname name first given name middle name email cutemoon94 gmail com gender u appointment number 4512 9081 9072 1138 date birth september 30 1994 test date august 24 2019 jeong moon kyung 53 8 daehyun dong queens house room 401 seodaemun gu seoul korea republic inst code dept code country birth korea republic native language korean test center stn13681d kaec test center country korea republic security identification id type passport id xxxxxxxxxxxxxxxxxxxxx3935 issuing country korea repub pdf score report downloaded printed test taker august 24 2019 test date scores mybesttm scores highest section scores valid test dates august 30 2019 sum highest total score reading reading 28 section scores 28 30 test date 30 aug 24 2019 108 108 listening test date 120 listening 30 120 30 30 aug 24 2019 120 speaking speaking 25 test date 25 30 30 aug 24 2019 writing 25 25 aug 24 2019 writing 30 test date 30 total score reported 

In [18]:
# 전처리한 데이터에서 각 단어를 인덱스로 벡터화 해야함.
# 모델에 따라 입력값의 길이가 동일해야 하기 때문에 일정 길이로 자르고 부족한 부분은 특정값으로 채우는 패딩과정을 진행해야 함

clean_train_df = pd.DataFrame({'text': clean_train_texts, 'label': train_data['label']})
clean_train_df

,text,label
0,ts toefl ibt test taker score report ets secur...,1
1,ets toefl bt test taker score report name moon...,1
2,ets toefl ibt test taker score report name bai...,1
3,ets toefl ibt test taker score report name jun...,1
4,ets toefl ibt test taker score report name par...,1
...,...,...
13573,제 n2020123316 호 졸업증명 서 성 명 김상온 생 년월 일 1985년 10...,1
13574,4 docu verification xd6a 4c8d c36e 41a2 2 oniv...,1
13575,원본대조번호 07a1 5d18 0719 2159 제 0272059 호 졸 업 증 명...,1
13576,0 0 01062044 108631201606000596 ar r e bttp ww...,1


In [19]:
# 토크나이저 모듈을 생성한 후 정제된 데이터에 적용하고 인덱스로 구성된 벡터로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_texts)
text_sequences = tokenizer.texts_to_sequences(clean_train_texts)

In [20]:
# 텍스트가 아닌 인덱스의 벡터로 구성됨을 확인
print(text_sequences[1])

[37, 126, 534, 11, 467, 10, 153, 130, 3103, 31993, 1261, 853, 1623, 130, 655, 597, 130, 1484, 130, 1264, 86334, 82, 137, 68, 624, 94, 1426, 218, 39629, 5883, 27403, 31994, 50, 208, 739, 185, 819, 11, 50, 1356, 66, 36, 3103, 31993, 205, 23996, 86335, 306, 10728, 11094, 205, 26, 163, 1439, 465, 1064, 465, 554, 208, 26, 163, 833, 180, 433, 11, 878, 27404, 8021, 9476, 18, 265, 11, 878, 554, 26, 163, 685, 1411, 402, 669, 1754, 402, 3000, 12899, 86336, 1545, 554, 26, 1902, 746, 10, 153, 960, 492, 11, 467, 1356, 66, 36, 11, 50, 187, 1538, 187, 1119, 302, 187, 278, 11, 1871, 1356, 172, 36, 1718, 1119, 86, 234, 1678, 66, 36, 162, 10, 86, 234, 302, 187, 56, 11, 50, 56, 1766, 1766, 109, 11, 50, 539, 539, 109, 56, 56, 56, 56, 1678, 66, 36, 539, 448, 448, 234, 11, 50, 234, 56, 56, 1678, 66, 36, 322, 322, 234, 234, 56, 11, 50, 56, 1678, 66, 36, 162, 10, 367, 308, 1470, 1708, 2051, 187, 1093, 1538, 2019, 80, 80, 1024, 78, 166, 118, 164, 729, 997, 37, 37, 238, 126, 126, 534, 193, 225, 166, 118, 164, 3

In [24]:
# 위와 같이 전체 데이터가 인덱스로 구성돼 있을 텐데 각 인덱스가 어떤 단어를 의미하는지 확인할수 있도록 단어 사전 확인
word_vocab = tokenizer.word_index
word_vocab["<PAD>"] = 0
print(word_vocab)

b646': 240517, 'iwb076807': 240518, '교교무처장교': 240519, 'oe89': 240520, '452b': 240521, '082e': 240522, 'cmeo06982': 240523, '언더우드국제': 240524, '22d0': 240525, '4ob0': 240526, '143호': 240527, '건국대2019학2543': 240528, '앱민원센터': 240529, '9b08': 240530, '69c3': 240531, '12ae': 240532, 'ethodis': 240533, '9fcf': 240534, 'ce15': 240535, '글로낼행정학': 240536, '출력옮': 240537, '114358': 240538, '3124900142023645': 240539, '170990': 240540, '3346432746543818': 240541, '9aae': 240542, '6cbc': 240543, '36601': 240544, '기계금속재료전공': 240545, '91e6': 240546, 'cme013446': 240547, 'yunhaeng': 240548, '178577호': 240549, '17학4803': 240550, 'bwww': 240551, '40e': 240552, 'walici': 240553, '30c8': 240554, '85f5': 240555, 'iwbo75668': 240556, '010951': 240557, '73cd': 240558, '231d': 240559, 'ae69': 240560, 'ec2f': 240561, 'cme0o1750': 240562, '원본대소가': 240563, '0028248': 240564, '바이오동물전공': 240565, '88450': 240566, '생물산업전문학사': 240567, '3035899774123665': 240568, '05c8': 240569, '32bc3': 240570, 'ef3b': 240571, '690af':

In [25]:
print("전체 단어 개수: ", len(word_vocab))

전체 단어 개수:  243268


In [26]:
# 단어 사전 뿐 아니라 전체 단어 개수도 이후 모델에서 사용되기 때문에 저장해둔다.
# 데이터에 대한 정보인 단어 사전과 전체 단어 개수는 새롭게 딕셔너리 값을 지정해서 저장해두자

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)

In [27]:
# 마지막 전처리
# 현재 각 데이터는 서로 길이가 다른데 이 길이를 하나로 통일해야 이후 모델에 바로 적용할수 있기 때문에
# 특정 길이를 최대 길이로 정하고 더 긴 데이터의 경우 뒷부분을 자르고 짧은 데이터의 경우에는 0 값으로 패딩하는 작업을 진행

MAX_SEQUENCE_LENGTH = 336  # 문장 최대 길이 -> 단어 개수 통계의 중간값

train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

print('Shape of train data: ', train_inputs.shape)

Shape of train data:  (13574, 336)


In [28]:
# 학습시 정답을 나타내는 값을 넘파이 배열로 저장
# 넘파이 배열로 변환하는 이유는 이후 전처리한 데이터를 저장할 때 넘파이 형태로 저장하기 때문
train_labels = np.array(train_data['label'])
print('Shape of label tensor:', train_labels.shape)

Shape of label tensor: (13574,)


In [41]:
DATA_IN_PATH = './data_in/'         # 저장할 데이터 경로
TRAIN_INPUT_DATA = 'train_input.npy' # 벡터화한 데이터
TRAIN_LABEL_DATA = 'train_label.npy' # 정답 라벨
TRAIN_CLEAN_DATA = 'train_clean.csv' # 정제된 텍스트 데이터
DATA_CONFIGS = 'data_configs.json'   # 데이터 정보(단어 사전, 전체 단어 개수)

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

In [42]:
# 전처리 된 데이터를 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

# 정제된 텍스트를 csv 형태로 저장
clean_train_df.to_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, index = False)


# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)

In [43]:
clean_train_df.isna().sum()

text     0
label    0
dtype: int64

In [44]:
clean_train_df['text'][5000]

'문서 확인번호 xd3c c978 1268 6b5c 성 적 증 명 서 발행번호 12020 021464 번 20141946 성 대 임유정 학 미술대학 생년월일 학과 학부 1996년 1월 26일 조소과 입학일자 졸업 일자 2014년 3월 1일 2019년 2월 21일 졸업 학점 성적 구분 교과목명 학점 성적 구분 교과목명 학점 성적 취득학점 21 평점평균 4 37 구분 교과목명 2014학년도 1학기 사고와 표현 effective english speech c 영상으로 떠나는 베트남 문화기행 전통과 평면 모델링 0 서양 조각사 인체표현 취득학점 17 평점평균 4 15 nnnn www ao b b ao 2016학년도 2학기 b 예술 비즈니스 모델 따라잡기 b 문학과 시대 읽기 c 우리 문화 속의 한자에 e 디지털 조형 2 e 현대조각 2 e 재료기법 2 취득학점 19 평점평균 4 36 000 b b b 2014학년도 2학기 정신인 effective written english b 경제학 개론 c 이슬람문화와 예술 입체와 공간 획과 여백 모델 표현 한국조각사 취득학점 19 평점평균 3 54 3 2 8 2 2018학년도 1학기 c 마음을읽어주는미술치료 c 직무적성검사 분석 e 자유창작1 e 창작세미나 1 취득학점 15 평점평균 4 15 nnnn wwwn p b b b b c b 2018학년도 2학기 e 자유창작 2 e 졸업 작품연구 졸업인증 졸업논문 취득학점 6 평점평균 4 20 4 2 0 0 ao ao p p 2015학년도 1학기 c tv드라마의 세계 컴퓨터와 조형 미술사 1 동시대 조각의 현장 1 인체소조 1 0 석조기법 1 취득학점 17 평점평균 3 72 1000000 b bo bt b 취득학점 132 평점평균 4 07 4 5 백분율 95 09 100 석 차 1 36 성적우수졸업 이 하 여백 2015학년도 2학기 c 스페인 문화와 예술 c 생활 프랑스어 1 c 직무분석 커리어 개발 디지털미디어 1 도조기법 인체소조 2 취득학점 18 평점평균 4 07 332226 b 

In [45]:
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [46]:
train_data.isna().sum()

text     0
label    0
dtype: int64

In [47]:
train_data

,text,label
0,ts toefl ibt test taker score report ets secur...,1
1,ets toefl bt test taker score report name moon...,1
2,ets toefl ibt test taker score report name bai...,1
3,ets toefl ibt test taker score report name jun...,1
4,ets toefl ibt test taker score report name par...,1
...,...,...
13569,제 n2020123316 호 졸업증명 서 성 명 김상온 생 년월 일 1985년 10...,1
13570,4 docu verification xd6a 4c8d c36e 41a2 2 oniv...,1
13571,원본대조번호 07a1 5d18 0719 2159 제 0272059 호 졸 업 증 명...,1
13572,0 0 01062044 108631201606000596 ar r e bttp ww...,1


In [49]:
# 테스트 데이터도 동일하게 처리
test_data = pd.read_excel("gradnet_test_v2.xlsx")

In [50]:
clean_test_texts = []
for text in test_data['text']:
    clean_test_texts.append(preprocessing(text, remove_stopwords = True))


clean_test_df = pd.DataFrame({'text': clean_test_texts, 'id': test_data['id']})
test_id = np.array(test_data['id'])

text_sequences = tokenizer.texts_to_sequences(clean_test_texts)
test_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [51]:
test_id

array([5976, 6063, 6195, ..., 3878, 5757, 1511], dtype=int64)

In [52]:
TEST_INPUT_DATA = 'test_input.npy'
TEST_CLEAN_DATA = 'test_clean.csv'
TEST_ID_DATA = 'test_id.npy'

np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(DATA_IN_PATH + TEST_ID_DATA, 'wb'), test_id)
clean_test_df.to_csv(DATA_IN_PATH + TEST_CLEAN_DATA, index = False)

In [53]:
clean_test_df.isna().sum()

text    0
id      0
dtype: int64

In [54]:
test_data = pd.read_csv( DATA_IN_PATH + TEST_CLEAN_DATA )

In [55]:
test_data.isna().sum()

text    0
id      0
dtype: int64

In [56]:
test_data['text']

0       ets toefl ibt toefl bt test taker score report...
1       ets toefl ibt test taker score report name lim...
2       ets toefl bt toefl bt test taker score report ...
3       ets toefl ibt test taker score report name jo ...
4       ets toefl ibt test taker score report name cha...
                              ...                        
3390    문서확인번호 xdcf 3794 f122 cbd6 교 학 1885 대 학위수 여증 명...
3391    californa university southern california trust...
3392    문서확인번호 xd85 c828 225a 5c89 서울여대중 제 2020 25922 ...
3393    문서확인번호 xdie 26c4 d928 5ef8 미 제 0100002k2011040...
3394    문서확인번호 xdfe 787d b60d 50b0 universin 1885 세 대 ...
Name: text, Length: 3395, dtype: object

# 모델 학습 및 테스트_Linear Regression with TF-IDF

In [76]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [77]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [78]:
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [79]:
train_data.isna().sum()

text     0
label    0
dtype: int64

In [80]:
train_data = train_data.dropna()  # 결측값 제거 안하면 다다음 코드에서 에러 발생

In [81]:
text = list(train_data['text'])
label = list(train_data['label'])

In [82]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(text)
y = np.array(label)

In [83]:
X

<13574x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 14320761 stored elements in Compressed Sparse Row format>

In [84]:
features = vectorizer.get_feature_names()

In [85]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [86]:
lgs = LogisticRegression(class_weight='balanced') 
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [87]:
predicted = lgs.predict(X_eval)

In [88]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.999632


In [89]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [90]:
test_data = test_data.dropna()

In [91]:
test_data

,text,id
0,ets toefl ibt toefl bt test taker score report...,5976
1,ets toefl ibt test taker score report name lim...,6063
2,ets toefl bt toefl bt test taker score report ...,6195
3,ets toefl ibt test taker score report name jo ...,6238
4,ets toefl ibt test taker score report name cha...,6123
...,...,...
3390,문서확인번호 xdcf 3794 f122 cbd6 교 학 1885 대 학위수 여증 명...,4233
3391,californa university southern california trust...,2815
3392,문서확인번호 xd85 c828 225a 5c89 서울여대중 제 2020 25922 ...,3878
3393,문서확인번호 xdie 26c4 d928 5ef8 미 제 0100002k2011040...,5757


In [92]:
testDataVecs = vectorizer.transform(test_data['text'])

In [93]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 1 1 ... 1 1 1]


In [94]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'label': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'tfidf_toeic.csv', index=False, quoting=3)

# 모델 학습 및 테스트_Random Forest

In [95]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer

In [96]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
TEST_SIZE = 0.2
RANDOM_SEED = 42

In [97]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [98]:
# 결측값 제거를 위해 판다스 데이터프레임으로 변환
train_data1 = pd.DataFrame(train_data)
train_data1

,text,label
0,ts toefl ibt test taker score report ets secur...,1
1,ets toefl bt test taker score report name moon...,1
2,ets toefl ibt test taker score report name bai...,1
3,ets toefl ibt test taker score report name jun...,1
4,ets toefl ibt test taker score report name par...,1
...,...,...
13569,제 n2020123316 호 졸업증명 서 성 명 김상온 생 년월 일 1985년 10...,1
13570,4 docu verification xd6a 4c8d c36e 41a2 2 oniv...,1
13571,원본대조번호 07a1 5d18 0719 2159 제 0272059 호 졸 업 증 명...,1
13572,0 0 01062044 108631201606000596 ar r e bttp ww...,1


In [99]:
# 결측값 제거
train_data = train_data1.dropna()

In [100]:
texts = list(train_data['text'])
y = np.array(train_data['label'])

In [101]:
vectorizer = CountVectorizer(analyzer = "word", max_features = 5000) 

train_data_features = vectorizer.fit_transform(texts)

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [104]:
from sklearn.ensemble import RandomForestClassifier


# 랜덤 포레스트 분류기에  100개 의사 결정 트리를 사용한다.
forest = RandomForestClassifier(n_estimators = 100) 

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다.
forest.fit( train_input, train_label )

RandomForestClassifier()

In [105]:
print("Accuracy: %f" % forest.score(eval_input, eval_label))  # 검증함수로 정확도 측정

Accuracy: 0.999632


In [106]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)
test_data

,text,id
0,ets toefl ibt toefl bt test taker score report...,5976
1,ets toefl ibt test taker score report name lim...,6063
2,ets toefl bt toefl bt test taker score report ...,6195
3,ets toefl ibt test taker score report name jo ...,6238
4,ets toefl ibt test taker score report name cha...,6123
...,...,...
3390,문서확인번호 xdcf 3794 f122 cbd6 교 학 1885 대 학위수 여증 명...,4233
3391,californa university southern california trust...,2815
3392,문서확인번호 xd85 c828 225a 5c89 서울여대중 제 2020 25922 ...,3878
3393,문서확인번호 xdie 26c4 d928 5ef8 미 제 0100002k2011040...,5757


In [107]:
test_data = test_data.dropna()

In [108]:
test_text = list(test_data['text'])
ids = list(test_data['id'])

In [109]:
test_data_features = vectorizer.transform(test_text)

In [110]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = forest.predict(test_data_features)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"label": result} )

# 이제 csv파일로 만든다.
output.to_csv( DATA_OUT_PATH + "bag_toeic.csv", index=False, quoting=3 )